In [ ]:
%load_ext autoreload
%autoreload 2

import h5py
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('presentation')

from shabanipy.jj.plotting_general import plot_vi_dr_curve
from shabanipy.jj.utils import extract_switching_current
from shabanipy.jj.iv_analysis import extract_critical_current, analyse_vi_curve

#: Name of the sample that must appear in the measurement name usually of the form "{Wafer}-{Piece}_{Design}-{Iteration}_{Junction}_{Cooldown}"
SAMPLE_NAME =
SAMPLE_ID =

#: hdf5 file number 
FILE_NUM =

#: Path to store generated files
PATH = (f"/Users/bh/Desktop/Code/Topological JJ/Samples/{SAMPLE_NAME}/{SAMPLE_ID}")

#: Name of generated processed data file
PROCESSED_DATA_NAME = (f"{PATH}/Data/{SAMPLE_ID}_processed-data-{FILE_NUM}.hdf5")

h = h5py.File(PROCESSED_DATA_NAME, 'r')

field_y = 'In-plane Field - Y::0'
field_z = 'In-plane Field - Z::0'

f = h['Data'][f'{field_y}&{field_z}']

In [ ]:
data = np.array(f["Voltage Drop"])
scaled_v_drop = np.array(f["ScaledVoltage"])
bias = np.array(f["Bias"])
dR = np.array(f["dR"])

In [ ]:
"""analyse_vi_curve: Voltage threshold in V above which the junction is not considered to carry a
supercurrent anymore. Used in the determination of the critical current. Usually of the order of a couple e-5 or e-4. 
Default is 1e-4."""
ic_voltage_threshold = 1e-4
"""extract_switching_current: Similar to above but lower in value.  Usually of the order of a couple e-5"""
switching_bias_threshold = 2.35e-5

"""Positive bias value above which the data can be used to extract the
normal resistance. Default is 10e-6."""
high_bias_threshold = 5e-6

x = analyse_vi_curve(bias,scaled_v_drop, ic_voltage_threshold, high_bias_threshold)
y = extract_switching_current(np.array([bias]),np.array([scaled_v_drop]),switching_bias_threshold)

print('Rn Positive: ',x[0], 'Ω')
print('Rn Negative: ',x[1], 'Ω')
print('analyse_vi_curve - Ic Positive: ',x[2]*1e6, 'µA')
print('analyse_vi_curve - Ic Negative: ',x[3]*1e6, 'µA')
print('extract_switching_current - Ic Positive: ',y*1e6, 'µA')
print('Ie Positive: ',x[4]*1e6, 'µA')
print('Ie Negative: ',x[5]*1e6, 'µA')

In [ ]:
plot_vi_dr_curve(bias,scaled_v_drop,dR,
                text = f'R$_n$ Positive: {np.round(x[0],0)}Ω \nR$_n$ Negative: {np.round(x[1],0)}Ω \n'\
                f'I$_c$ Positive: {np.round(x[2]*1e6,2)}µA \nI$_c$ Negative: {np.round(x[3]*1e6,2)}µA\n'\
                f'I$_c$R$_n$ Positive: {np.round(int(x[0])*x[2]*1e6,2)}µV \nI$_c$R$_n$ Negative: {np.round(int(x[1])*x[3]*1e6,2)}µV\n'\
                f'I$_e$ Positive: {np.round(x[4]*1e6,2)}µA \nI$_e$ Negative: {np.round(x[5]*1e6,2)}µA',
#                 savgol_windowl = 3, savgol_polyorder = 1,
#                 bias_limits = ,
#                 fig_size = ,
                 
                )
plt.savefig(f'Figs/Characterization/Characterization-map_{SAMPLE_ID}_field-y:{field_y[16:]}_field-z:{field_z[16:]}_{FILE_NUM}.pdf', bbox_inches = 'tight')